In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

In [165]:
#read in df with dummies
former_final_df = pd.read_csv('../data/final_df.csv')

C:\Users\Conrad\AppData\Local\Temp\ipykernel_24404\279734686.py:2: DtypeWarning: Columns (5,6,9,10,12,13,14,15,16,17,18,19,21,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  former_final_df = pd.read_csv('../data/final_df.csv')


In [166]:
# Set view options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [167]:
former_final_df = former_final_df[former_final_df['Aftertreatment System'].notna()]

In [168]:
almost_final_df = former_final_df.drop(former_final_df.loc[:,'AcceleratorPedal':'TurboBoostPressure'], axis = 1)
final_df = almost_final_df.drop(['EventTimeStamp', 'ESS_Id', 'fmi'], axis = 1)
final_df

,EquipmentID,spn,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,DAS Module to ECM Communication,ECU Power Output Supply Voltage,Electric Lift Pump,Electrical Charging System Voltage,Electronic Fuel Injection Control Valve,Electronic Throttle Control,Engine Air Filter,Engine Brake,Engine Charge Air Cooler Outlet Temperature,Engine Control Module,Engine Coolant,Engine Cooling System,Engine Diesel Particulate Filter,Engine Exhaust Back Pressure,Engine Fuel,Engine Injector Metering Rail,Engine Intake Air Heater,Engine Misfire,Engine Oil,Engine Position Sensor,Engine Protection Torque Full Derate,Engine Protection Torque Partial Derate,Engine Speed,Engine Starter Motor Relay,Engine Total Idle Hours,Exhaust Gas,Exhaust Gas Recirculation System,Fan Control,Fuel Control Module,High Pressure Common Rail Fuel Pressure Relief Valve,Indicator Lamp,Injector Metering Rail,Injector Solenoid,Intake Manifold,Intake Throttle,J1939 Network,Real Time Clock,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
1,301,596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,301,3226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,301,639,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,301,596,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,301,639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554847,2377,1231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
554848,2377,1231,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
554850,2377,111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
554851,2377,111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [169]:
#Get list of unique EquipmentIDs
eq_ids = final_df.drop_duplicates(subset='EquipmentID')
eq_ids = eq_ids[['EquipmentID']]
eq_ids.shape[0]

1026

In [170]:
#create a column of random numbers from 0-1
np.random.seed(321)

eq_ids['random'] = (np.random.randint(0, 10000, eq_ids.shape[0]))/10000

eq_ids

,EquipmentID,random
1,301,0.6682
433,302,0.4220
2050,303,0.5929
2787,304,0.9617
4024,305,0.8360
...,...,...
554816,2371,0.1721
554821,2374,0.3518
554834,2375,0.0887
554839,2376,0.5903


In [171]:
#create column 'modeling' labeling ~35% of the data for training, ~30% for validation, and ~35% for testing 
eq_ids['modeling'] = np.where(((eq_ids.random <= 0.35)), 'training', np.where(((eq_ids.random <= 0.65)), 'validation', 'testing'))

In [172]:
#merge modeling column onto original dataframe
final_df = final_df.sort_values(by='EquipmentID')
eq_ids = eq_ids.sort_values(by='EquipmentID')

final_df = final_df.merge(eq_ids, on='EquipmentID')

In [173]:
#created training dataframe
df_training = final_df[final_df['modeling']=='training']
df_training = df_training.drop(columns=['modeling', 'random'])
df_training

,EquipmentID,spn,Aftertreatment System,Ambient Air Temperature,Anti-Lock Braking (ABS) Controller,Auxiliary Input/Output,Auxiliary Temperature Sensor Input Circuit,Barometric Pressure Sensor,Battery,Crankcase,DAS Module to ECM Communication,ECU Power Output Supply Voltage,Electric Lift Pump,Electrical Charging System Voltage,Electronic Fuel Injection Control Valve,Electronic Throttle Control,Engine Air Filter,Engine Brake,Engine Charge Air Cooler Outlet Temperature,Engine Control Module,Engine Coolant,Engine Cooling System,Engine Diesel Particulate Filter,Engine Exhaust Back Pressure,Engine Fuel,Engine Injector Metering Rail,Engine Intake Air Heater,Engine Misfire,Engine Oil,Engine Position Sensor,Engine Protection Torque Full Derate,Engine Protection Torque Partial Derate,Engine Speed,Engine Starter Motor Relay,Engine Total Idle Hours,Exhaust Gas,Exhaust Gas Recirculation System,Fan Control,Fuel Control Module,High Pressure Common Rail Fuel Pressure Relief Valve,Indicator Lamp,Injector Metering Rail,Injector Solenoid,Intake Manifold,Intake Throttle,J1939 Network,Real Time Clock,Sensor Supply,Steering Wheel Angle Sensor,Total Vehicle Distance,Transmission Communications Failure Idle Torque Limit,Transmission Oil,Turbocharger,Unknown,Variable Geometry Turbocharger,Vehicle Identification Number,Wheel-Based Vehicle Speed
6490,306,829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6491,306,829,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6492,306,96,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6493,306,829,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6494,306,639,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545692,2371,111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545704,2375,2863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
545705,2375,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
545706,2375,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [175]:
df_sampling = df_training

df_sampling['spn'] = df_sampling['spn'].mask(df_sampling['spn'] == 1569, 'partial')
df_sampling['spn'] = df_sampling['spn'].mask(df_sampling['spn'] == 1569, 'partial')
df_sampling['spn'].unique()

array([829, 96, 639, 3216, 3226, 5246, 'partial', 4094, 1028, 248, 596,
       1231, 84, 627, 5743, 5848, 97, 1761, 3482, 171, 1483, 157, 3490,
       3361, 641, 111, 50353, 4364, 1209, 3251, 2863, 4096, 51923, 651,
       1045, 25780, 0, 110, 654, 245, 630, 100, 175, 3242, 3246, 3936,
       723, 256, 4276, 101, 576, 17096, 3253, 3720, 4794, 444, 3218, 1481,
       43088, 1024, 35527, 768, 17590, 42190, 3031, 4342, 1675, 3228,
       5397, 791, 102, 611, 81, 655, 3556, 1059, 2791, 5298, 789, 790,
       3703, 3698, 3697, 3360, 4363, 3610, 4765, 1328, 652, 105, 3480,
       917, 184, 563, 614, 3362, 3058, 1808, 5019, 1068, 1067, 5394, 656,
       1807, 792, 794, 629, 5319, 3509, 27, 793, 512, 3584, 29902, 47284,
       36017, 2866, 4376, 4796, 5491, 3514, 39093, 4334, 3597, 168, 412,
       653, 1172, 1347, 5024, 862, 3363, 1323, 247, 9295, 32894, 520201,
       612, 801, 802, 523531, 4344, 4340, 524033, 524037, 647, 4346, 4360,
       803, 190, 523543, 520203, 810, 108, 3060, 1127, 62

In [115]:
#create dataframe of both training and testing to have large test dataset for testing. 
#With size of dataset, probably can just keep it confined to modeling == 'testing'
df_train_test = final_df[final_df['modeling']!='validation']
df_train_test = df_train_test.drop(columns=['modeling', 'random'])
df_train_test.shape

(397175, 82)

In [116]:
#final dataframe without modeling or random column for final predictions
df_total = final_df.drop(columns=['modeling', 'random'])

In [117]:
#setting up for model testing with training dataset. Realized here about how may need to adjust target variable 
#to implement oversampling
features = df_total.iloc[:,2:].columns.tolist()

target =['spn']

X_train = df_training[features]
y_train = df_training[target]

In [121]:
logistic = LogisticRegression()

In [122]:
#unsucessful logistic regression fit because of discussed NaNs
logreg = logistic.fit(X_train, y_train)

C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Conrad\anaconda3\envs\p310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [124]:
df_test = final_df[final_df['modeling']=='testing']
df_test = df_test.drop(columns=['modeling','random'])
X_test = df_test[features]
y_test = df_test[target]

In [126]:
y_pred = logreg.predict(X_test)

print(accuracy_score(y_test,y_pred))
print(f1_score(y_test, y_pred, average = 'weighted'))

0.6507381877428249
0.5685694768342724
